# PROJET : 
# RECONNAISSANCE VOCALE
# SYSTEME DE TRADUCTION ADAPTE AUX LUNETTES CONNECTEES  
  
    
      
***

Dans l'itération précédente, nous avons procédé à une traduction mot à mot en utilisant un ensemble de phrases traduites à la fois en anglais et en français et en se servant de la place des mots dans la phrase. Nous avions constaté que cette méthode n'était pas probante.

Au cours de cette étape, nous avons procédé en entrainant un algorithme de deep learning qui se base sur la correspondance entre l'espace vectoriel de mots source vers les mots cibles. Grâce aux mots transparents entre les deux langues, l'algorithme peut générer une matrice de correspondance permettant la traduction de l'ensemble des mots.

   
      
   
## Partie II : Modélisation - Itération 2  
   
   


Cette partie reprend le dictionnaire de référence tel que travaillé précédemment. 

In [1]:
import numpy as np
import pandas as pd

# Dataset utilisé : DS2
data = pd.read_csv("en-fr.txt", sep = " ", names = ["Anglais", "Français"])
data.head()

,Anglais,Français
0,the,le
1,the,les
2,the,la
3,and,et
4,was,fut


In [2]:
uniq_eng = list(data.Anglais.unique())
print("Nbre de mots anglais uniques dans le 2ème dataset :", len(uniq_eng), "mots.\n")

uniq_fr = list(data.Français.unique())
print("Nbre de mots français uniques dans le 2ème dataset :", len(uniq_fr), "mots.")

Nbre de mots anglais uniques dans le 2ème dataset : 94680 mots.

Nbre de mots français uniques dans le 2ème dataset : 97034 mots.


In [3]:
# Création du dictionnaire de référence (DS2) : association de toutes les traductions françaises possibles pour chaque mot
#anglais en clé
data_ord = data.groupby(['Anglais']).agg(lambda x : list(x)).reset_index()
dico_ref = {data_ord['Anglais'][i] : data_ord['Français'][i] for i in range(data_ord.shape[0])}

### 1. Création des dictionnaires de vectorisation des mots anglais et français  

Cette partie de code sert à créer les dictionnaires de vectorisation des mots français et anglais contenus dans notre dataset de référence (DS2) à partir de matrices d'embeddings pré-entrainées.

En raison de la durée d'exécution, le code suivant ne sera exécuté qu'une seule fois : la suite des analyses se base sur les fichiers csv créés dans cette partie.

In [4]:
# Téléchargement des matrices d'embedding pré-entrainées 
"""
from pathlib import Path
from urllib.request import urlretrieve

PATH_TO_DATA = Path()

fr_embeddings_path = PATH_TO_DATA / 'cc.fr.300.vec.gz'
if not fr_embeddings_path.exists(): 
    urlretrieve('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz', fr_embeddings_path)
en_embeddings_path = PATH_TO_DATA / 'cc.en.300.vec.gz'
if not en_embeddings_path.exists():
    urlretrieve('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz', en_embeddings_path)
"""

"\nfrom pathlib import Path\nfrom urllib.request import urlretrieve\n\nPATH_TO_DATA = Path()\n\nfr_embeddings_path = PATH_TO_DATA / 'cc.fr.300.vec.gz'\nif not fr_embeddings_path.exists(): \n    urlretrieve('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz', fr_embeddings_path)\nen_embeddings_path = PATH_TO_DATA / 'cc.en.300.vec.gz'\nif not en_embeddings_path.exists():\n    urlretrieve('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz', en_embeddings_path)\n"

La cellule suivante reprend le code de création de la classe Word2Vec() : 

+ Les méthodes d'initialisation de la classe et de loading permettent, à partir des matrices téléchargées précédemment, de générer un array de 2 millions de lignes (nombre de mots inclus dans les matrices pré-entrainées) et de 301 colonnes, représentant le mot et le vecteur de taille 300 correspondant, dont découlent les éléments suivants : 

    + Le mapping word/index et index/word (méthodes word2id et id2word)
    
    + La méthode words et la méthode embeddings : chaque ligne de la matrice est splittée après le 1er élément, constituant ainsi une liste des mots de la matrice (méthode words) et un array de 2 millions de lignes et de 300 colonnes représentant chaque vecteur (méthode embeddings)
    
+ La méthode encode retourne pour le mot inclus en attribut l'array du vecteur correspondant si ce mot appartient à la matrice pré-entrainée, sinon il retourne un array zéros de longeur 300.

In [23]:
import gzip

class Word2Vec():
    def __init__(self, filepath):
        self.words, self.embeddings = self.load_wordvec(filepath)
        # Mappings for O(1) retrieval:
        self.word2id = {word: idx for idx, word in enumerate(self.words)}
        self.id2word = {idx: word for idx, word in enumerate(self.words)}
    
    def load_wordvec(self, filepath):
        assert str(filepath).endswith('.gz')
        words = []
        embeddings = []
        with gzip.open(filepath, 'rt', encoding = 'utf-8') as f:
            next(f)  # Skip header
            for i, line in enumerate(f):
                word, vec = line.split(',', 1)
                words.append(word)                                 
                embeddings.append(np.fromstring(vec, sep=','))     
        print('Loaded %s pretrained word vectors' % (len(words)))
        return words, np.vstack(embeddings)
    
    def encode(self, word):
        '''
        Inputs:
        -word : mot
        
        Output:
        - l'embedding du mot correspondant
        '''
        if word in self.words : 
            row = self.embeddings[self.word2id[word],:]
            return(row)
        
        else : 
            return(np.zeros((1, self.embeddings.shape[1])))  #si le mot n'est pas dans le dictionnaire
    


In [ ]:
# Chargement des données et création des mappings word/vector
#fr_word2vec = Word2Vec(filepath=fr_embeddings_path)
#en_word2vec = Word2Vec(filepath=en_embeddings_path)

In [ ]:
# Création des dictionnaires de vectorisation des mots du dataset 2
#dico_eng = {}
#for i in uniq_eng:
    #if i in en_word2vec.words:
        #dico_eng.update({i : en_word2vec.encode(i)})
        
#dico_fr = {}
#for i in uniq_fr:
    #if i in fr_word2vec.words:
        #dico_fr.update({i : fr_word2vec.encode(i)})

In [ ]:
# Création d'un fichier reprenant les dictionnaires incluant les vectorisations des mots anglais et français du DS2
#pd.DataFrame.from_dict(data = dico_eng, orient = 'index').to_csv("dico_eng.csv", header = False)
#pd.DataFrame.from_dict(data = dico_fr, orient = 'index').to_csv("dico_fr.csv", header = False)

   
### 2. Matrice de correspondance W  et traduction des mots anglais 
   
   

##### Chargement des données des dictionnaires de vectorisation

In [5]:
# Lecture des dictionnaires de vectorisations des mots anglais et français
df_eng = pd.read_csv("dico_eng.csv", header= None, index_col = 0)
dico_eng = {df_eng.index[i] : np.array(df_eng.iloc[i, :]) for i in range(df_eng.shape[0])}

df_fr = pd.read_csv("dico_fr.csv", header= None, index_col = 0)
dico_fr = {df_fr.index[i] : np.array(df_fr.iloc[i, :]) for i in range(df_fr.shape[0])}

In [6]:
# Création de la liste des mots anglais vectorisés
dico_eng_words = list(dico_eng.keys())

# Création de la liste des vecteurs correspondants
dico_eng_embeds = []
for i in dico_eng.keys():
    dico_eng_embeds.append(dico_eng[i])

In [7]:
# Création de la liste des mots français vectorisés
dico_fr_words = list(dico_fr.keys())

# Création de la liste des vecteurs correspondants
dico_fr_embeds = []
for i in dico_fr.keys():
    dico_fr_embeds.append(dico_fr[i])

##### Matrice de correspondance W

Une fois que nous avons obtenu les vectorisations de tous les mots en communs des deux jeux de données, nous allons créer deux matrices d'embeddings X et Y contenant les vectorisations de tous les mots transparents apparaissant dans les deux langues. 

Le but sera ensuite de trouver la matrice W qui  projettera l'espace vectoriel des mots sources (Anglais dans notre cas) sur l'espace vectoriel des mots cibles (Français dans notre cas) afin que les mots ayant les mêmes significations dans les deux langues aient des coordonnées les plus proches possibles. 

Afin de trouver la matrice W, nous allons utiliser une formule reposant sur l'orthogonalité et les propriétés d'un matrice:
<center> $W^* = UV^T$ avec $U$$\Sigma$$V^T$ $=$ $SVD$ $(YX^T)$ </center>

Ou SVD est la décomposition en valeur singulière d'une matrice.

In [8]:
# Création de la liste des mots transparents (identiques dans les dictionnaires français et anglais)
mots_transparents = [word for word in dico_eng if word in dico_fr]
print("Nbre de mots transparents :", len(mots_transparents), "mots.")

Nbre de mots transparents : 33072 mots.


In [9]:
# Encodage des mots transparents
X, Y = np.empty([300,len(mots_transparents)]),np.empty([300,len(mots_transparents)])
for i, word in enumerate(mots_transparents): 
        X[:,i] = dico_eng[word]
        Y[:,i] = dico_fr[word]
        
assert X.shape[0] == 300 and Y.shape[0] == 300

In [10]:
# Calcul de W, la matrice de correspondance entre l'anglais et le français
U, sigma, Vtranspose = np.linalg.svd(Y.dot(X.T))
W = U.dot(Vtranspose)   

##### Fonction de traduction

Nous allons créer une fonction qui permet d'associer à tout mot anglais les k mots français les plus similaires (k représentant le nombre de traductions proposées par le dictionnaire de référence). On calcule pour cela la métrique cosine similarity que l'on cherche à maximiser afin de proposer la meilleure traduction possible.

In [11]:
# Création de la fonction de traduction de l'anglais au français
def get_closest_french_words(eng_word):
        '''
        Inputs:
        - eng_word : mot en anglais
        
        Output:
        -renvoie les k mots les plus proches dans la traduction
        '''
        # k représente le nombre de mots proposés en traduction du mot anglais dans le dictionnaire de référence
        k = len(dico_ref[eng_word])
        
        # On reprend le vecteur du mot anglais sélectionné
        eng_obj = dico_eng[eng_word]
        
        # On projette le mot anglais dans l'espace vectoriel des mots français grâce la matrice de correspondance W
        aligne_eng = W.dot(eng_obj.T)
        
        # On crée l'array reprenant tous les vecteurs des mots français
        fr_embeds = np.array(dico_fr_embeds)
        
        # Calcul de la similitude du mot anglais avec les mots français avec la métrique cosine similarity
        norm_prod = np.linalg.norm(aligne_eng)*np.linalg.norm(fr_embeds, axis=1) 
        scores = fr_embeds.dot(aligne_eng) / norm_prod 
        
        # Récupération des k meilleurs scores de similitude
        best_k = np.flip(np.argsort(scores))[:k]
        
        # Liste contenant une liste des k mots français les plus proches du mot anglais et la liste contenant leur degré de similitude
        return [[dico_fr_words[idx] for idx in best_k], [scores[idx] for idx in best_k]]

In [12]:
# Exemple de traduction
get_closest_french_words('car')[0]

['voiture', 'véhicule']

In [ ]:
# Création du dictionnaire de traduction qui affecte à chaque mot anglais, les k traductions possibles
#from tqdm import tqdm
#dico_trad = {}
#for word in tqdm(dico_eng_words):
    #dico_trad.update({word : get_closest_french_words(word)})

In [ ]:
# Création d'un fichier csv pour recharger le dictionnaire obtenu
#pd.DataFrame.from_dict(data = dico_trad, orient = 'index').to_csv("dico_trad.csv", header = False)

### 3. Scoring

Une fois notre dictionnaire de traduction créé grâce au word embedding, nous allons ensuite le comparer au dictionnaire de traduction de référence.

Le score représentera le pourcentage de similarité entre les traductions des deux dictionnaires. 

In [13]:
# Import du dataframe contenant le dictionnaire de traduction
df_trad_imp = pd.read_csv("dico_trad.csv", header = None, index_col = 0)

In [14]:
# Retraitement du dataframe précédent
carac = [",", "'", "[", "]"]
for i in range(df_trad_imp.shape[0]):
    for c in df_trad_imp.iloc[i, 0]:
        if c in carac:
            df_trad_imp.iloc[i, 0] = df_trad_imp.iloc[i, 0].replace(c, " ")
    df_trad_imp.iloc[i, 0] = df_trad_imp.iloc[i, 0].split()

In [15]:
# Création du dataframe de scoring

df_trad = df_trad_imp.reset_index().iloc[:, 0:2]
df_trad.columns = ['Mot_anglais', 'Trad_func']

In [16]:
df_ref = pd.DataFrame(list(dico_ref.items()), columns = ['Mot_anglais', 'Trad_ref'])

In [17]:
df_score = pd.merge(df_ref, df_trad)

In [18]:
# Calcul du score : rapport entre le nbre de traductions communes (dictionnaire de référence et 
#dictionnaire de traduction créé) et le nbre de traductions de référence
score = []
tot = 0
res = 0
for i in range(df_score.shape[0]):
    for j in df_score.Trad_func[i]:
        if j in df_score.Trad_ref[i]:
            tot +=1
        res = round(tot / len(df_score.Trad_ref[i]) * 100, 2)
    tot = 0
    score.append(res)

df_score['Score'] = score
df_score.head()

,Mot_anglais,Trad_ref,Trad_func,Score
0,aaa,[aaa],[lbl],0.0
1,aaaa,[aaaa],[aaaaaa],0.0
2,aaaaaa,[aaaaaa],[aaaaaa],100.0
3,aab,[aab],[jaan],0.0
4,aac,[aac],[xvid],0.0


In [19]:
score_model = df_score.Score.mean()
print("Le sore est de :", score_model)

Le sore est de : 46.90854424855996


##### Utilisation d'une méthode utilisant un k fixe =5

In [ ]:
#Création d'un dictionnaire comportant les mots français et leurs vecteurs associés
#for idx, i in enumerate(tqdm(uniq_fr)):
#    if i in fr_word2vec.words:
#        dico_fr.update({i : fr_word2vec.encode(i)})

In [ ]:
##Création d'un dictionnaire comportant les mots anglais et leurs vecteurs associés
#for idx, i in enumerate(tqdm(uniq_eng)):
#    if i in en_word2vec.words:
#        dico_eng.update({i : en_word2vec.encode(i)})

In [ ]:
#On exporte les deux dictionnaires en fichier csv afin de pouvoir les réutiliser, leur création ayant pris pas mal de temps
#pd.DataFrame.from_dict(data = dico_eng, orient = 'index').to_csv("dico_eng.csv", header = False)
#pd.DataFrame.from_dict(data = dico_fr, orient = 'index').to_csv("dico_fr.csv", header = False)

In [ ]:
#A partir des deux fichiers csv compressés en format gzip, nous créons deux objets de la classe word2vec
fr_word2vec2 = Word2Vec(filepath='dico_fr.csv.gz', vocab_size=2000000)
en_word2vec2 = Word2Vec(filepath='dico_eng.csv.gz', vocab_size=2000000)

In [26]:
# Obtenir les mots qui apparaissent dans les 2 vocabulaires (mots qui ont des chaines de caractères identiques)
#mots_transparents = [word for word in fr_word2vec2.words if word in en_word2vec2.words]

# On encode nos mots : obtention des embeddings de chaque mot
#X, Y = np.empty([300,len(mots_transparents)]),np.empty([300,len(mots_transparents)])
#for i, word in enumerate(mots_transparents) : 
#        X[:,i] = en_word2vec2.encode(word)
#        Y[:,i] = fr_word2vec2.encode(word)
#        
#assert X.shape[0] == 300 and Y.shape[0] == 300

In [ ]:
#Calcul de la matrice W
U, sigma, Vtranspose = np.linalg.svd(Y.dot(X.T))
W = U.dot(Vtranspose)        
W

In [ ]:
#définition d'une fonction de traduction
"""def get_closest_french_words(en_word, k):
        en_obj = en_word2vec2.encode(en_word)
        aligne_en = W.dot(en_obj.T)
        fr_embeds = fr_word2vec2.embeddings
        norm_prod = np.linalg.norm(aligne_en)*np.linalg.norm(fr_embeds, axis=1) 
        scores = fr_embeds.dot(aligne_en) / norm_prod 
        best_k = np.flip(np.argsort(scores))[:k]
        return ([fr_word2vec2.words[idx] for idx in best_k], [scores[idx] for idx in best_k])
    
get_closest_french_words('cat', 5)"""

In [ ]:
#création d'un dictionnaire de traduction Anglais: 5 mots français les plus probable
#from tqdm import tqdm
#dico_trad = {}
#for word in tqdm(en_word2vec2.words):
#    dico_trad.update({word : get_closest_french_words(word, 5)})

In [ ]:
#récupération des mots d'origines et mots cibles en ignorant les probabilités 
"""dico_trad2 = {}
for word in dico_trad.keys():
    dico_trad2.update({word : dico_trad[word][0]})"""

In [ ]:
#création d'un dataframe à partie du dictionnaire dico_trad2
#df_trad = pd.DataFrame(list(dico_trad2.items()), columns = ['Mot_anglais', 'Trad_func'])
#df_trad.head()

In [ ]:
#data_ord = data.groupby(['Anglais']).agg(lambda x : list(x)).reset_index()
#dico_ref = {data_ord['Anglais'][i] : data_ord['Français'][i] for i in range(data_ord.shape[0])}

In [ ]:
#df_trad_csv = df_trad.to_csv('df_trad.csv')

In [24]:
df_trad2 = pd.read_csv('df_trad.csv', index_col = 0)

In [25]:
carac = [",", "'", "[", "]"]
for i in range(df_trad2.shape[0]):
    for c in df_trad2.iloc[i, 1]:
        if c in carac:
            df_trad2.iloc[i, 1] = df_trad2.iloc[i, 1].replace(c, " ")
    df_trad2.iloc[i, 1] = df_trad2.iloc[i, 1].split()

In [27]:
df_ref = pd.DataFrame(list(dico_ref.items()), columns = ['Mot_anglais', 'Trad_ref'])
df_score = pd.merge(df_ref, df_trad2)

In [28]:
score = []
tot = 0
res = 0
for i in range(df_score.shape[0]):
    for j in df_score.Trad_func[i]:
        if j in df_score.Trad_ref[i]:
            tot +=1
    res = round(tot / len(df_score.Trad_ref[i]) * 100, 2)
    tot = 0
    score.append(res)

df_score['Score'] = score
df_score.head()

,Mot_anglais,Trad_ref,Trad_func,Score
0,aaa,[aaa],"[lbl, aaa, aem, cbg, aik]",100.0
1,aaaa,[aaaa],"[aaaaaa, nnn, lbl, ksk, kah]",0.0
2,aaaaaa,[aaaaaa],"[aaaaaa, eef, lbl, ksk, okai]",100.0
3,aab,[aab],"[jaan, aln, aib, aab, mnc]",100.0
4,aac,[aac],"[xvid, aac, wma, aif, sacd]",100.0


In [29]:
score_model = df_score.Score.mean()
score_model

59.221875972191356

Avec les deux scoring, nous obtenons les scores de 46.90% et 59.22%. Ces scores ne sont pas très bons alors même que les matrices d'embedding sélectionnées ont été pré-entraînées pendant longtemps. 

Nous ne validons pas cette méthode de traduction mot à mot. 

Nous allons continuer par un système de traduction phrase à phrase (Seq2Seq).